In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = (SparkSession
        .builder
        .appName("SparkSQLdepartures")
        .getOrCreate())

In [3]:
csv = "C:/Users/sean.cornillie/Education/LearningSparkV2/Spark_Dev/datasets/departuredelays.csv"

In [4]:
### Source our csv and then create a temp table so we can execute sql code
df = (spark.read.format("csv")
         .option("inferSchema", "true")
         .option("header","true")
         .load(csv))

df.createOrReplaceTempView("us_delay_flights_tbl")

In [5]:
df.show(n=10, truncate=False)

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|6    |602     |ABE   |ATL        |
|1020600|-8   |369     |ABE   |DTW        |
|1021245|-2   |602     |ABE   |ATL        |
|1020605|-4   |602     |ABE   |ATL        |
|1031245|-4   |602     |ABE   |ATL        |
|1030605|0    |602     |ABE   |ATL        |
|1041243|10   |602     |ABE   |ATL        |
|1040605|28   |602     |ABE   |ATL        |
|1051245|88   |602     |ABE   |ATL        |
|1050605|9    |602     |ABE   |ATL        |
+-------+-----+--------+------+-----------+
only showing top 10 rows



In [6]:
### Basic SQL example. Functions just like reqular query would.
spark.sql("""SELECT distance
                ,origin
                ,destination
             FROM us_delay_flights_tbl
             WHERE 1=1
                 AND distance > 1000
             ORDER BY distance desc""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [7]:
### More basic SQL querying.
spark.sql("""SELECT date
                ,delay
                ,origin
                ,destination
             FROM us_delay_flights_tbl
             WHERE 1=1
                 AND origin = 'SFO'
                 AND destination = 'ORD'
                 AND delay > 120
             ORDER BY delay desc""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



#### Exercise: Convert Date to readable format and find days that the SFO/ORD delays were most common.

In [8]:
### Converting the date with specified 'MM/dd/yyyy' format threw errors, omitting timestamp method worked just fine (below).
new_df = df.withColumn("departuredate", to_timestamp(col("date"))).drop("date")

new_df.show(n=10)

+-----+--------+------+-----------+-------------------+
|delay|distance|origin|destination|      departuredate|
+-----+--------+------+-----------+-------------------+
|    6|     602|   ABE|        ATL|1970-01-12 09:54:05|
|   -8|     369|   ABE|        DTW|1970-01-12 12:30:00|
|   -2|     602|   ABE|        ATL|1970-01-12 12:40:45|
|   -4|     602|   ABE|        ATL|1970-01-12 12:30:05|
|   -4|     602|   ABE|        ATL|1970-01-12 15:27:25|
|    0|     602|   ABE|        ATL|1970-01-12 15:16:45|
|   10|     602|   ABE|        ATL|1970-01-12 18:14:03|
|   28|     602|   ABE|        ATL|1970-01-12 18:03:25|
|   88|     602|   ABE|        ATL|1970-01-12 21:00:45|
|    9|     602|   ABE|        ATL|1970-01-12 20:50:05|
+-----+--------+------+-----------+-------------------+
only showing top 10 rows



In [9]:
### Then need to replace our temp table so we can run some SQL code
new_df.createOrReplaceTempView("us_delay_flights_tbl")

In [10]:
### Aggregate avg delay by day of week
spark.sql("""SELECT date_format(departuredate, 'EEEE') as DayOfWeek
                ,avg(delay) as AvgDelay
             FROM us_delay_flights_tbl
             WHERE 1=1
                 AND origin = 'SFO'
                 AND destination = 'ORD'
             GROUP BY DayOfWeek
             ORDER by AvgDelay desc""").show(10)

+---------+------------------+
|DayOfWeek|          AvgDelay|
+---------+------------------+
|   Monday|23.425249169435215|
|   Sunday|21.653465346534652|
| Saturday| 16.79179810725552|
|  Tuesday| 11.92972972972973|
|   Friday|              11.5|
| Thursday|11.494117647058824|
|Wednesday| 9.346774193548388|
+---------+------------------+



In [11]:
### Aggregated number of delays by day of week. 'EEEE' pulls the dayofweek datepart out of the timestamp.
spark.sql("""SELECT date_format(departuredate, 'EEEE') as DayOfWeek
                ,count(delay) as NumDelays
             FROM us_delay_flights_tbl
             WHERE 1=1
                 AND origin = 'SFO'
                 AND destination = 'ORD'
                 AND delay > 0
             GROUP BY DayOfWeek
             ORDER by NumDelays desc""").show(10)

+---------+---------+
|DayOfWeek|NumDelays|
+---------+---------+
| Saturday|      167|
| Thursday|      156|
|   Monday|      155|
|   Sunday|      127|
|Wednesday|      111|
|  Tuesday|       94|
|   Friday|       92|
+---------+---------+



In [12]:
### Alternatively use spark built in function 'dayofweek' instead of converting the date.
spark.sql("""SELECT dayofweek(departuredate) as DayOfWeek
                ,count(delay) as NumDelays
             FROM us_delay_flights_tbl
             WHERE 1=1
                 AND origin = 'SFO'
                 AND destination = 'ORD'
                 AND delay > 0
             GROUP BY DayOfWeek
             ORDER by NumDelays desc""").show(10)

+---------+---------+
|DayOfWeek|NumDelays|
+---------+---------+
|        7|      167|
|        5|      156|
|        2|      155|
|        1|      127|
|        4|      111|
|        3|       94|
|        6|       92|
+---------+---------+



#### More complicated case when example

In [13]:
spark.sql("""SELECT delay
                ,origin
                ,destination
                ,CASE
                    WHEN delay > 360 THEN 'Very Long Delays'
                    WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                    WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                    WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                    WHEN delay = 0 THEN 'No Delays'
                    ELSE 'Early'
                END as Flight_Delays
             FROM us_delay_flights_tbl
             ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



#### Exercise: Convert the SQL queries above to use the DataFrame API

In [14]:
#spark.sql("""SELECT distance
#                ,origin
#                ,destination
#             FROM us_delay_flights_tbl
#             WHERE 1=1
#                 AND distance > 1000
#             ORDER BY distance desc""").show(10)

(new_df
    .select("distance", "origin", "destination")
    .where(col("distance") > 1000)
    .sort(desc("distance"))
    .show(10)
)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [15]:
#spark.sql("""SELECT date
#                ,delay
#                ,origin
#                ,destination
#             FROM us_delay_flights_tbl
#             WHERE 1=1
#                 AND origin = 'SFO'
#                 AND destination = 'ORD'
#                 AND delay > 120
#             ORDER BY delay desc""").show(10)

(new_df
    .select("departuredate", "origin", "destination")
    .where(col("origin") == "SFO")
    .where(col("destination") == 'ORD')
    .where(col("delay") > 120)
    .show(10))

+-------------------+------+-----------+
|      departuredate|origin|destination|
+-------------------+------+-----------+
|1970-01-12 09:56:50|   SFO|        ORD|
|1970-01-12 12:58:50|   SFO|        ORD|
|1970-01-12 12:43:30|   SFO|        ORD|
|1970-01-13 10:56:50|   SFO|        ORD|
|1970-01-14 11:48:45|   SFO|        ORD|
|1970-01-15 01:45:10|   SFO|        ORD|
|1970-01-15 18:36:40|   SFO|        ORD|
|1970-01-12 09:53:57|   SFO|        ORD|
|1970-01-12 15:44:18|   SFO|        ORD|
|1970-01-12 15:38:40|   SFO|        ORD|
+-------------------+------+-----------+
only showing top 10 rows



### Creating SQL Databases & Tables

In [16]:
### First we'll create the database itself
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

DataFrame[]

#### Creating a Managed Table

In [ ]:
### Method 1: Issues a SQL statement
spark.sql("CREATE TABLE managed_us_delay_flights_tbl (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

In [ ]:
### Method 2: Using the DataFrame API

csv = 